### OCI Data Science - Useful Tips
Everything stored in the <span style="background-color: #d5d8dc ">/home/datascience</span> folder is now stored on your block volume drive. The <span style="background-color: #d5d8dc ">ads-examples</span> folder has moved outside of your working space. Notebook examples are now accessible through a Launcher tab "Notebook Examples" button.
<details>
<summary><font size="2">1. Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">2. OCI Configuration and Key Files Set Up</font></summary><p>Follow the instructions in the getting-started notebook. That notebook is accessible via the "Getting Started" Launcher tab button.</p>
</details>
<details>
<summary><font size="2">3. Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">4. Typical Cell Imports and Settings</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import MLData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">5. Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import logging
import os
import cx_Oracle
#logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
#from ads.dataset.factory import DatasetFactory
#from sqlalchemy import create_engine

In [2]:
# Add TNS_ADMIN to the environment
os.environ['TNS_ADMIN'] = "/home/datascience/ADB"
connection = cx_Oracle.connect('hackathon', 'bCfrCC0981267', 'adw_low')

In [164]:
QUERY = """

WITH
T_MES AS
(
	
	select
		CPF_CRIP,
		ANOMES,
        COUNT( DISTINCT CPF_CRIP) QTD_CLIENTES_DISTINTOS
	from
		EVE_BUFFER_CCI_MANU_HCKT T1
	where	
		ANOMES = '202009'

	GROUP BY
		CPF_CRIP,
		ANOMES
	
),

T_MES_1 AS
(
	
	select
		CPF_CRIP,
		ANOMES
	from
		EVE_BUFFER_CCI_MANU_HCKT T1
	where	
		ANOMES = '202008'

	GROUP BY
		CPF_CRIP,
		ANOMES
	
),

T_MES_2 AS
(
	
	select
		CPF_CRIP,
		ANOMES
	from
		EVE_BUFFER_CCI_MANU_HCKT T1
		
	where	
		ANOMES = '202007'

	GROUP BY
		CPF_CRIP,
		ANOMES
	
),

T_MES_3 AS
(
	
	select
		CPF_CRIP,
		ANOMES
	from
		EVE_BUFFER_CCI_MANU_HCKT T1
		
	where	
		ANOMES = '202006'

	GROUP BY
		CPF_CRIP,
		ANOMES
	
),

T_MES_MENOR AS
(
	
	select
		CPF_CRIP,
		MIN(ANOMES) ANOMES
	from
		EVE_BUFFER_CCI_MANU_HCKT T1
		
	GROUP BY
		CPF_CRIP
	
)

SELECT
    NU_ANOMES_REFEREN,
    DS_TIPO_RECORRENCIA,
    QTD_CLIENTES,
    (QTD_CLIENTES / (SUM(QTD_CLIENTES) OVER (PARTITION BY NU_ANOMES_REFEREN)))  AS PERCENTUAL
FROM
(
    SELECT
        NU_ANOMES_REFEREN,
        DS_TIPO_RECORRENCIA,
        COUNT(*) QTD_CLIENTES
    FROM
    (
        SELECT
            T_MES.CPF_CRIP,
            T_MES.ANOMES 		AS NU_ANOMES_REFEREN,
            T_MES_1.ANOMES 		AS NU_ANOMES_RECORR1,
            T_MES_2.ANOMES 		AS NU_ANOMES_RECORR2,
            T_MES_3.ANOMES 		AS NU_ANOMES_RECORR3,
            CASE
            WHEN T_MES_1.ANOMES IS NOT NULL AND T_MES_2.ANOMES IS NOT NULL AND T_MES_3.ANOMES IS NOT NULL THEN '1 - RECORRENTE  3M'
            WHEN T_MES_1.ANOMES IS NOT NULL AND T_MES_2.ANOMES 								  IS NOT NULL THEN '2 - RECORRENTE  2M'
            WHEN T_MES_1.ANOMES IS NOT NULL 												 			  THEN '3 - RECORRENTE  1M' 
            WHEN T_MES_1.ANOMES = T_MES_MENOR.ANOMES
            THEN '5 - CLIENTE NOVO'
            ELSE '4 - RECORRENTE ALTERNADO'
            END AS DS_TIPO_RECORRENCIA

        FROM
            T_MES

            LEFT JOIN T_MES_1 ON
                T_MES_1.CPF_CRIP = T_MES.CPF_CRIP

            LEFT JOIN T_MES_2 ON
                T_MES_2.CPF_CRIP = T_MES.CPF_CRIP

            LEFT JOIN T_MES_3 ON
                T_MES_3.CPF_CRIP = T_MES.CPF_CRIP

            LEFT JOIN T_MES_MENOR ON
                T_MES_MENOR.CPF_CRIP = T_MES.CPF_CRIP
    )

    GROUP BY
        NU_ANOMES_REFEREN,
        DS_TIPO_RECORRENCIA
    ORDER BY
        NU_ANOMES_REFEREN,
        DS_TIPO_RECORRENCIA
) TT
	
"""	
	

In [165]:
TESTE = pd.read_sql(QUERY, con=connection)


In [168]:
TESTE.style.format({'PERCENTUAL':"{:.2%}"})

,NU_ANOMES_REFEREN,DS_TIPO_RECORRENCIA,QTD_CLIENTES,PERCENTUAL
0,202009,1 - RECORRENTE 3M,129030,39.10%
1,202009,2 - RECORRENTE 2M,32643,9.89%
2,202009,3 - RECORRENTE 1M,52080,15.78%
3,202009,4 - RECORRENTE ALTERNADO,116248,35.23%


In [167]:
TESTE.to_csv (r'Clientes_Recorrentes.csv', index = False, header=True)

In [211]:
QUERY_TICKET = """

SELECT 
	ANOMES,
	sum(TICU_VL_TOTALITEM) AS TICU_VL_TOTALITEM,
	COUNT( DISTINCT CPF_CRIP) AS TOTAL_ITENS,
	SUM(TICU_VL_TOTALITEM) / COUNT( DISTINCT CPF_CRIP) TICKET_MEDIO 
FROM 
	EVE_BUFFER_CCI_MANU_HCKT 
group by
	ANOMES
order by	
	ANOMES


"""



In [212]:
Ticket_Medio = pd.read_sql(QUERY_TICKET, con=connection)


In [213]:
Ticket_Medio.style.format({'TICU_VL_TOTALITEM': 'R${:,.2f}','TOTAL_ITENS':'{:,.0f}','TICKET_MEDIO': 'R${:,.2f}'})

,ANOMES,TICU_VL_TOTALITEM,TOTAL_ITENS,TICKET_MEDIO
0,202003,"R$115,918,976.59","324,828",R$356.86
1,202004,"R$112,326,891.31","301,338",R$372.76
2,202005,"R$125,326,631.53","307,818",R$407.15
3,202006,"R$117,219,854.17","307,200",R$381.58
4,202007,"R$119,477,909.74","316,743",R$377.21
5,202008,"R$134,191,550.08","325,726",R$411.98
6,202009,"R$121,195,924.66","330,001",R$367.26


In [229]:
QUERY_TICKET_AQUISICAO = """

SELECT 
	substr(ICUP_DT_REF,1,6) ANOMES,
	sum(ICUP_VL_TOTALITEM) AS TICU_VL_TOTALITEM,
	COUNT( DISTINCT CPF_CRIP) AS TOTAL_ITENS,
	SUM(ICUP_VL_TOTALITEM) / COUNT( DISTINCT CPF_CRIP) TICKET_MEDIO 
FROM 
	EVE_BUFFER_CCI_AQUISICAO_HCKT 
where
    substr(ICUP_DT_REF,1,6) >= 202001
group by
	substr(ICUP_DT_REF,1,6)
order by	
	substr(ICUP_DT_REF,1,6)



"""


In [230]:
Ticket_Medio_Aquisicao = pd.read_sql(QUERY_TICKET_AQUISICAO, con=connection)

In [232]:
Ticket_Medio_Aquisicao.style.format({'TICU_VL_TOTALITEM': 'R${:,.2f}','TOTAL_ITENS':'{:,.0f}','TICKET_MEDIO': 'R${:,.2f}'})

,ANOMES,TICU_VL_TOTALITEM,TOTAL_ITENS,TICKET_MEDIO
0,202001,"R$54,064,151.80","174,000",R$310.71
1,202002,"R$58,841,025.82","182,347",R$322.69
2,202003,"R$33,782,452.89","119,629",R$282.39
3,202004,"R$72,167,594.77","167,390",R$431.13
4,202005,"R$86,812,068.75","179,259",R$484.28
5,202006,"R$79,413,694.30","177,069",R$448.49
6,202007,"R$78,035,430.87","183,606",R$425.02
7,202008,"R$48,873,488.75","136,462",R$358.15
8,202009,"R$26,314,595.46","93,872",R$280.32


In [354]:
QUERY_CONTA_ATIVA = """



select 
	T_MOB.DATE_REF,
	T_MOB.CACCSERNO_CRIPT,
	T_MOB.DMSC_NU_MOB,
    T_MOB.DMSC_FG_CONTAATIVDA,
    T_MOB.DMSC_FG_CONTAATIV,
    T_MOB.DMSC_FG_CONTAAPTA,
	SUM(CASE WHEN T3.TAUT_DH_TRANSACAO IS NOT NULL THEN 1 ELSE 0 END) QTD_TRANSACAO
from 
(

	SELECT 
		DATE_REF,
		CACCSERNO_CRIPT,
		DMSC_NU_MOB,
		DMSC_FG_CONTAATIVDA,
		DMSC_FG_CONTAATIV,
		DMSC_FG_CONTAAPTA
	FROM
		EVE_SUMARIOCONTA_MANU_HCKT
	WHERE	
		ROWNUM = 1
	AND CACCSERNO_CRIPT IS NOT NULL
	AND DMSC_FG_CONTAATIVDA = 1
	AND DMSC_FG_CONTAATIV   = 1
	AND DMSC_FG_CONTAAPTA   = 1


	UNION ALL

	SELECT 
		DATE_REF,
		CACCSERNO_CRIPT,
		DMSC_NU_MOB,
		DMSC_FG_CONTAATIVDA,
		DMSC_FG_CONTAATIV,
		DMSC_FG_CONTAAPTA
	FROM
		EVE_SUMARIOCONTA_MANU_HCKT
	WHERE	
		ROWNUM = 1
	AND DMSC_FG_CONTAATIVDA = 1
	AND DMSC_FG_CONTAATIV   = 1
	AND DMSC_FG_CONTAAPTA   = 0
	AND CACCSERNO_CRIPT IS NOT NULL

	UNION ALL

	SELECT 
		DATE_REF,
		CACCSERNO_CRIPT,
		DMSC_NU_MOB,
		DMSC_FG_CONTAATIVDA,
		DMSC_FG_CONTAATIV,
		DMSC_FG_CONTAAPTA
	FROM
		EVE_SUMARIOCONTA_MANU_HCKT
	WHERE	
		ROWNUM = 1
	AND DMSC_FG_CONTAATIVDA = 1
	AND DMSC_FG_CONTAATIV   = 0
	AND DMSC_FG_CONTAAPTA   = 0
	AND CACCSERNO_CRIPT IS NOT NULL

	UNION ALL

	SELECT 
		DATE_REF,
		CACCSERNO_CRIPT,
		DMSC_NU_MOB,
		DMSC_FG_CONTAATIVDA,
		DMSC_FG_CONTAATIV,
		DMSC_FG_CONTAAPTA
	FROM
		EVE_SUMARIOCONTA_MANU_HCKT
	WHERE	
		ROWNUM = 1
	AND DMSC_FG_CONTAATIVDA = 0
	AND DMSC_FG_CONTAATIV   = 1
	AND DMSC_FG_CONTAAPTA   = 1
	AND CACCSERNO_CRIPT IS NOT NULL

	UNION ALL

	SELECT
		DATE_REF,
		CACCSERNO_CRIPT,
		DMSC_NU_MOB,
		DMSC_FG_CONTAATIVDA,
		DMSC_FG_CONTAATIV,
		DMSC_FG_CONTAAPTA
	FROM
		EVE_SUMARIOCONTA_MANU_HCKT
	WHERE	
		ROWNUM = 1
	AND DMSC_FG_CONTAATIVDA = 0
	AND DMSC_FG_CONTAATIV   = 0
	AND DMSC_FG_CONTAAPTA   = 1
	AND CACCSERNO_CRIPT IS NOT NULL


	UNION ALL

	SELECT
		DATE_REF,
		CACCSERNO_CRIPT,
		DMSC_NU_MOB,
		DMSC_FG_CONTAATIVDA,
		DMSC_FG_CONTAATIV,
		DMSC_FG_CONTAAPTA
	FROM
		EVE_SUMARIOCONTA_MANU_HCKT
	WHERE	
		ROWNUM = 1
	AND DMSC_FG_CONTAATIVDA = 0
	AND DMSC_FG_CONTAATIV   = 0
	AND DMSC_FG_CONTAAPTA   = 0
	AND CACCSERNO_CRIPT IS NOT NULL

) T_MOB

	
	INNER JOIN EVE_CHAVES_MANU_HCKT T2 ON
		T2.CACCSERNO_CRIPT = T_MOB.CACCSERNO_CRIPT
	
	INNER JOIN EVE_AUTHORIZATIONS_MANU_HCKT T3 ON
		T2.NUMBERX_CRIPT = T3.NUMBERX_CRIPT
	
GROUP BY
	T_MOB.DATE_REF,
	T_MOB.CACCSERNO_CRIPT,
	T_MOB.DMSC_NU_MOB,
    T_MOB.DMSC_FG_CONTAATIVDA,
    T_MOB.DMSC_FG_CONTAATIV,
    T_MOB.DMSC_FG_CONTAAPTA,
	CASE WHEN T3.TAUT_DH_TRANSACAO IS NOT NULL THEN 1 ELSE 0 END
"""

In [355]:
CONTA_ATIVA = pd.read_sql(QUERY_CONTA_ATIVA, con=connection)

In [356]:
CONTA_ATIVA

,DATE_REF,CACCSERNO_CRIPT,DMSC_NU_MOB,DMSC_FG_CONTAATIVDA,DMSC_FG_CONTAATIV,DMSC_FG_CONTAAPTA,QTD_TRANSACAO
0,202009,C5AD7D,131,1,1,1,385
1,202005,B38700,126,0,0,1,71
2,202009,B2FDB4,130,0,0,0,48
3,202003,B2FDB4,124,0,1,1,48
4,202003,7D7A49,122,1,1,0,108


In [358]:
QUERY_MOB = """


SELECT 
	CASE WHEN DMSC_NU_MOB <= 6 						 THEN '1| <=6M        '
		 WHEN DMSC_NU_MOB > 6  AND DMSC_NU_MOB <= 12 THEN '2| >  6 <= 12 M'
		 WHEN DMSC_NU_MOB > 12 AND DMSC_NU_MOB <= 24 THEN '3| > 12 <= 24 M'
		 WHEN DMSC_NU_MOB > 24 AND DMSC_NU_MOB <= 30 THEN '4| > 24 <= 30 M'
		 WHEN DMSC_NU_MOB > 30                       THEN '5| > 30M'
	END FAIXA_MOB,
    DMSC_FG_CONTAATIVDA,
    DMSC_FG_CONTAATIV,
    DMSC_FG_CONTAAPTA,
    COUNT(*) QTD
FROM
    EVE_SUMARIOCONTA_MANU_HCKT

GROUP BY
	CASE WHEN DMSC_NU_MOB <= 6 						 THEN '1| <=6M        '
		 WHEN DMSC_NU_MOB > 6  AND DMSC_NU_MOB <= 12 THEN '2| >  6 <= 12 M'
		 WHEN DMSC_NU_MOB > 12 AND DMSC_NU_MOB <= 24 THEN '3| > 12 <= 24 M'
		 WHEN DMSC_NU_MOB > 24 AND DMSC_NU_MOB <= 30 THEN '4| > 24 <= 30 M'
		 WHEN DMSC_NU_MOB > 30                       THEN '5| > 30M'
	END,
    DMSC_FG_CONTAATIVDA,
    DMSC_FG_CONTAATIV,
    DMSC_FG_CONTAAPTA
ORDER BY
	CASE WHEN DMSC_NU_MOB <= 6 						 THEN '1| <=6M        '
		 WHEN DMSC_NU_MOB > 6  AND DMSC_NU_MOB <= 12 THEN '2| >  6 <= 12 M'
		 WHEN DMSC_NU_MOB > 12 AND DMSC_NU_MOB <= 24 THEN '3| > 12 <= 24 M'
		 WHEN DMSC_NU_MOB > 24 AND DMSC_NU_MOB <= 30 THEN '4| > 24 <= 30 M'
		 WHEN DMSC_NU_MOB > 30                       THEN '5| > 30M'
	END,
    DMSC_FG_CONTAATIVDA,
    DMSC_FG_CONTAATIV,
    DMSC_FG_CONTAAPTA

"""

In [359]:
MOB = pd.read_sql(QUERY_MOB, con=connection)

In [360]:
MOB

,FAIXA_MOB,DMSC_FG_CONTAATIVDA,DMSC_FG_CONTAATIV,DMSC_FG_CONTAAPTA,QTD
0,1| <=6M,0,0,0,82785
1,1| <=6M,0,0,1,1036330
2,1| <=6M,0,1,0,146402
3,1| <=6M,0,1,1,395540
4,1| <=6M,1,1,0,59576
5,1| <=6M,1,1,1,1592757
6,2| > 6 <= 12 M,0,0,0,252216
7,2| > 6 <= 12 M,0,0,1,592842
8,2| > 6 <= 12 M,0,1,0,129669
9,2| > 6 <= 12 M,0,1,1,378997
